In [29]:
from PyPDF2 import PdfReader
import re,fitz
import os
import pandas as pd

In [30]:
class EnemAutomata:
    def __init__(self):
        self.state = 0
        self.question = {}
        self.state_dict = {
            0:'header',
            1:'body',
            'A':'alternative A',
            'B':'alternative B',
            'C':'alternative C',
            'D':'alternative D',
            'E':'alternative E',
            'R':'Essay'
        }
    def clear_memory(self):
        self.state = 0
        self.question = {}
        
    def letter_state(self,current_state,next_state,part):
        if part.strip() != next_state:
            if current_state not in self.question:
                self.question[current_state] =''
            self.question[current_state]+=(part.strip('\n')+' ').replace('  ',' ')
        else:
            self.state = next_state
        
    def read(self,part):
        #final state
        if self.state=='E' and (re.search('questão [\d]+',part.lower()) or re.search('ENDOFENEM',part) or re.search('\*.*\*',part)):
            ret = self.question.copy()
            self.clear_memory()
            return ret
        elif re.search('\*.*\*',part.lower()) and self.state == 'R':
            ret = self.question.copy()
            self.clear_memory()
            return ret
        
        if not part:
            return False
        elif re.search('questão [\d]+',part.lower()) and self.state == 0:
            self.question['question'] = part.strip()
            self.state = 1
        elif re.search('instruções para a redação',part.lower()) and self.state == 0:
            self.question['question'] = 'redação'
            self.question['body'] = ''
            self.question['A'] = None
            self.question['B'] = None
            self.question['C'] = None
            self.question['D'] = None
            self.question['E'] = None
            self.state = 'R'
        elif self.state =='R':
            self.question['body']+=(part.strip('\n')+' ').replace('  ',' ')
        elif self.state == 1 and part.strip() != 'A':
            if 'body' not in self.question:
                self.question['body'] = ''
            self.question['body']+=(part.strip('\n')+' ').replace('  ',' ')
        elif self.state==1:
            self.state = 'A'
        elif self.state != 0 and self.state != 1:
            self.letter_state(self.state,chr(ord(self.state)+1),part)
        return False            

In [31]:
class PhysicalEnemParser:
    def __init__(self,enem_object):
        self.enem_object = enem_object
        parts = []
        for page_num in range(1,len(enem_object)):
            page = enem_object[page_num]
            image_list = page.get_images(full=True)
            to_remove = []
            for image in image_list:
                bbox = page.get_image_bbox(image)
                tb = page.get_textbox(bbox)
                to_remove.extend(tb.split('\n'))
            page_text = page.get_text().split('\n')
            for text in page_text:
                if text not in to_remove:
                    parts.append(text)
        self.parts = parts
    def parse_questions(self):
        self.automata = EnemAutomata()
        questions=[]
        for part in self.parts:
            accept = self.automata.read(part)
            while accept:
                questions.append(accept)
                accept = self.automata.read(part)
        return questions

In [66]:
files = os.listdir('./microdados_enem_2022/PROVAS E GABARITOS')
files = [file for file in files if ('laranja' in file.lower()) and ('cad' in file.lower())]
for file in files:
    enem = fitz.open('./microdados_enem_2022/PROVAS E GABARITOS/'+file)
    parser = PhysicalEnemParser(enem)
    questions = parser.parse_questions()
    df = pd.DataFrame(questions)
    essay_instructions = df.loc[df['question']=='redação','body']
    if not essay_instructions.empty:
        essay_instructions = essay_instructions.iloc[0]
        try:
            df.loc[df['question']=='redação','body'] = re.sub(r'PROPOSTA DE REDAÇÃO.*?(?!(TEXTO))','',essay_instructions) + re.search(r'PROPOSTA DE REDAÇÃO.*?(?=TEXTO)',essay_instructions).group(0)
        except:
            pass
    df.to_csv(f'./Data/{file.rstrip(".pdf")}.csv',index=False)

In [67]:
df = pd.read_csv('./Data/ENEM_2022_P1_CAD_09_DIA_1_LARANJA_LEDOR.csv')

In [68]:
df.loc[df['question']=='redação','body'].iloc[0]

'1. O rascunho da redação deve ser feito no espaço apropriado. 2. O texto definitivo deve ser escrito à tinta preta, na folha própria, em até 30 linhas. 3. A redação que apresentar cópia dos textos da Proposta de Redação ou do Caderno de Questões terá o número de linhas copiadas desconsiderado para a contagem de linhas. 4. Receberá nota zero, em qualquer das situações expressas a seguir, a redação que:  4.1. tiver até 7 (sete) linhas escritas, sendo considerada “texto insuficiente”;  4.2. fugir ao tema ou não atender ao tipo dissertativo-argumentativo;  4.3. apresentar parte do texto deliberadamente desconectada do tema proposto;  4.4. apresentar nome, assinatura, rubrica ou outras formas de identificação no espaço destinado ao texto. TEXTO 1 Você sabe quais são as comunidades e os povos tradicionais brasileiros? Talvez indígenas e quilombolas sejam os primeiros que passam pela cabeça, mas, na verdade, além deles, existem 26 reconhecidos oficialmente e muitos outros que ainda não foram